In [ ]:
import numpy as np
import math as m

# Base Level        

class GameInfo:
    default_beta = default_initial_mean/6.0
    default_draw_probability = 0.00
    default_dynamics_factor = default_initial_mean/300.0
    default_initial_mean = 25.0
    default_initial_standard_deviation = default_initial_mean/3.0
    
    def GameInfo(mean, standard_deviation, beta, dynamics_factor, draw_probability):
        self.mean = mean
        self.standard_deviation = standard_deviation
        self.beta = beta
        self.dynamics_factor = dynamics_factor
        self.draw_probability = draw_probability
    
    def update_beta (self, beta):
        self.beta = beta
    
    def update_draw_probability (self, draw_probability):
        self.draw_probability = draw_probability

    def update_dynamics_factor (self, dynamics_factor):
        self.dynamics_factor = dynamics_factor
        
    def update_mean (self, mean):
        self.mean = mean
    
    def update_standard_deviation (self, standard_deviation):
        self.standard_deviation = standard_deviation

        
# Rankings and attribute get/set to be implemented
    
    def __init__(self):
        return GameInfo(default_initial_mean, default_initial_standard_deviation, default_beta, default_dynamics_factor, default_draw_probability)

# Partial play implementation

class gaussian_distribution:
    
    def at (x, mean, standard_deviation):
        multiplier = 1/(standard_deviation * np.sqrt(2 * m.pi))
        exp_part = np.exp(-1* ((x - mean) ** 2)) / (2 * (standard_deviation ** 2))
        result = mulitplier * exp_part
        return result
    
    def cumulative_to (x, mean, standard_deviation):
        invsqrt2 = -0.707106781186547524400844362104
        result = ErrorFunctionCumulativeTo(invsqrt2 * x)
        return 0.5 * result
    
    def cumulative_to_value(x):
        return cumulative_to(x, 0, 1)
    
    def error_function_cumulative_to(x):
        z = abs(x)
        
        t = 2 / (2 + z)
        ty = 4 * t - 2
        
        coefficients = [ -1.3026537197817094, 6.4196979235649026e-1, 1.9476473204185836e-2, -9.561514786808631e-3, -9.46595344482036e-4, 
                        3.66839497852761e-4, 4.2523324806907e-5, -2.0278578112534e-5, -1.624290004647e-6, 1.303655835580e-6, 1.5626441722e-8, 
                        -8.5238095915e-8, 6.529054439e-9, 5.059343495e-9, -9.91364156e-10, -2.27365122e-10, 9.6467911e-11, 2.394038e-12, 
                        -6.886027e-12, 8.94487e-13, 3.13092e-13, -1.12708e-13, 3.81e-16, 7.106e-15, -1.523e-15, -9.4e-17, 1.21e-16, -2.8e-17]
        
        d = 0.0
        dd = 0.0
        
        for i in range(len(coefficients)):
            tmp = d
            d = ty * d - dd + coefficients[i]
            dd = tmp
        
        ans = t * np.exp(-z * z + 0.5*(coefficients[0] + ty * d) - dd)
        if x >= 0.0:
            return ans
        else:
            return 2 - ans
    
    
    def from_precision_mean (self, precision_mean, precision):
        self.precision = precision
        self.precision_mean = precision_mean
        self.variance = 1/precision
        self.standard_deviation = np.sqrt(self.variance)
        self.mean = precision_mean/precision
        return self
    
    def gaussian_abs_difference (self, left, right):
        gaussians = list(left.precision_mean - right.precision_mean, np.sqrt(abs(left.precision - right.precision)))
        return max(gaussians)
    
    def gaussian_division (self, numerator, denominator):
        return from_precision_mean(numerator.PrecisionMean - denominator.PrecisionMean, numerator.Precision - denominator.Precision)
    
    def gaussian_multiplication (self, left, right):
        return from_precision_mean(left.precision_mean + right.precision_mean, left.precision + right.precision)
    
    def inverse_cumulative_to(x, mean, standard_deviation):
        return mean - np.sqrt(2) * standard_deviation * inverse_error_function_cumulative_to(2 * x)
    
    def inverse_cumulative_to_value(x):
        return inverse_cumulative_to(x, 0, 1)
    
    def inverse_error_function_cumulative_to(p):
        if p >= 2:
            return -100
        elif p <= 0:
            return 100
        elif p > 1:
            pp = 2 - p
        else:
            pp = p
        
        t = np.sqrt(-2 * np.log(pp / 2))
        x = -0.70711 * ((2.30753 + t * 0.27061)/(1.0 + t * (0.99229 + t * 0.04481)) - t)
        
        for i in range(2):
            err = error_function_cumulative_to(x) - pp
            x += err / (1.12837916709551257 * np.exp(-(x * x)) - x * err)
        
        if p < 1:
            return x
        else:
            return -x
    
    def log_product_normalization (self, left, right):
        if left.precision == 0 or right.precision == 0:
            return 0
        variance_sum = left.variance + right.variance
        mean_difference = left.mean - right.mean
        
        log_sqrt_2pi = np.log(np.sqrt(2 * m.pi))
        
        return -log_sqrt_2pi - (np.log(variance_sum) / 2) - (mean_difference ** 2) / (2 * variance_sum))
    
    def log_ratio_normalization (self, numerator, denominator):
        if numerator.precision == 0 or denominator.precision == 0:
            return 0
        variance_difference = denominator.variance + numerator.variance
        mean_difference = numerator.mean - denominator.mean
        
        log_sqrt_2pi = np.log(np.sqrt(2 * m.pi))
        
        return np.log(denominator.variance) + log_sqrt_2pi - np.log(variance_difference) / 2 + (mean_difference ** 2) / (2 * variance_difference)
    
    def __init__(self, mean, standard_deviation):
        self.mean = mean
        self.standard_deviation = standard_deviation
        self.variance = standard_deviation * standard_deviation
        self.precision = 1/self.variance
        self.precision_mean = self.precision * mean

class rating:
    
    conservative_standard_deviation_multiplier = 3
    
    def get_conservative_rating (self):
        return self.mean - (self.conservative_standard_deviation_multiplier * self.standard_deviation)
    
    # First two arguments are previous instances of the class
    def get_partial_update(prior, full_posterior, update_percentage):
        prior_gaussian = gaussian_distribution(prior.mean, prior.standard_deviation)
        posterior_gaussian = gaussian_distribution(full_posterior.mean, full_posterior.standard_deviation)
        prior_precision = 1/(prior.standard_deviation * prior.standard_deviation)
        posterior_precision = 1/(full_posterior.standard_deviation * full_posterior.standard_deviation)
        prior_precision_mean = prior_precision * prior.mean
        posterior_precision_mean = posterior_precision * full_posterior.mean
        
        precision_difference = posterior_precision - prior_precision
        partial_precision_difference = update_percentage * precision_difference
        
        precision_mean_difference = posterior_precision_mean - prior_precision_mean
        partial_precision_mean_difference = precision_mean_difference * update_percentage
        
        partial_posterior_gaussian = gaussian_distribution.from_precision_mean(prior_precision_mean + partial_precision_mean_difference, prior_precision + partial_precision_difference)
        
        return rating(partial_posterior_gaussian.mean, partial_posterior_gaussian.standard_deviation, prior.conservative_standard_deviation_multiplier)
        
    def get_rating (self):
        return (self.mean, self.standard_deviation, self.conservative_standard_deviation_multiplier)
    
    def update_rating (self, mean, standard_deviation, conservative_standard_deviation_multiplier):
        self.mean = mean
        self.standard_deviation = standard_deviation
        self.conservative_standard_deviation_multiplier = conservative_standard_deviation_multiplier
    
    def __init__(self, mean, standard_deviation):
        self.mean = mean
        self.standard_deviation = standard_deviation
        self.conservative_standard_deviation_multiplier = conservative_standard_deviation_multiplier

class player:

    def update_d (self, partial_d_play):
        self.partial_d_play = partial_d_play
    
    def update_o (self, partial_o_play):
        self.partial_o_play = partial_o_play
    
    def __init__(self, id_no, partial_o_play, partial_d_play):
        self.id_no = id_no
        self.partial_o_play = partial_o_play
        self.partial_d_play = partial_d_play
        
class rank_sorter:
    
    def sort(teams, team_ranks):
        last_observed_rank = 0
        need_to_sort = False
        
        for i in range(len(teams)):
            if teams[i].rank < last_observed_rank:
                need_to_sort = True
                break
            last_observed_rank = teams[i].rank
        
        for i in range(len(teams)):
            

'''
class skill_calculator:
    
    def __init__(self, supported_options, total_teams_allowed, players_per_team_allowed):
        self.supported_options = supported_options
        self.total_teams_allowed = total_teams_allowed
        self.players_per_team_allowed = players_per_team_allowed
'''

class team:
    
    def add_player(self, player, rating):
        self.players[player] = rating
    
    def __init__(self, name):
        self.name = name
        self.players = {}
        self.rank = 0
        
class teams:
    
    def add_teams(self, team):
        self.teams.append((team, 0))
        
    def sort(self):
        last_observed_rank = 0
        need_to_sort = False
        
        for i in range(len(teams)):
            if self[i].rank < last_observed_rank:
                need_to_sort = True
                break
            last_observed_rank = self[i].rank
    
        if need_to_sort == False:
            return
        
        self.sort(key = self.teams[1])
    
    def __init__(self):
        self.teams = []
    
        

In [ ]:
# Factor Graph

class factor:
    
    def factor_name(self, name):
        self.name = name
    

class factor_list:
    
    def add_factor(self, a_factor):
        self.list.append(a_factor)
        return a_factor
    
    def log_normalization(self):
        for a_factor in self.list:
            a_factor.reset_marginals()
        sum_log_z = 0.0
        
        for i in range(len(self.list)):
            f = self.list[i]
            for j in range(f.number_of_messages)
                sum_log_z = sum_log_z + f.send_message(j)
            
        sum_log_s = 0
        
        for i in range(len(self.list)):
            sum_log_s = sum_log_s + self.list[i].acc + self.list[i].fac.log_normalization
        
        return sum_log_z + sum_log_s
            
    
    def __init__(self, factors):
        self.list = list(factors)

class message:
    
    def get_value(self):
        return self.value
    
    def set_value(self, value):
        self.value = value
    
    def __init__(self, value):
        self.value = value
        
class schedule:
    
    def visit():
        return visit(-1, 0)
    
    def __init__(self, name):
        self.name = name

class schedule_loop:
    
    def visit(self, depth, max_depth):
        total_iterations = 1
        delta = self.schedule_to_loop.visit(depth + 1, max_depth)
        while delta > self.max_delta:
            delta = self.schedule_to_loop.visit(depth + 1, max_depth)
            total_iterations += 1
        return delta
    
    def __init__(self, name, schedule_to_loop, max_delta):
        self.schedule_to_loop = schedule_to_loop
        self.max_delta = max_delta
        self.schedule = schedule(name)
        
class schedule_sequence:
    
    def visit(self, depth, maxdepth):
        max_delta = 0
        for schedule in self.schedules:
            max_delta = max(schedule.visit(depth + 1, max_depth), max_depth)
        return max_delta
    
    def __init__(self, name, schedules):
        self.name = name
        self.schedules = list(schedules)
    
class schedule_step:
    
    def visit(self, depth, maxdepth):
        delta = self.factor.update_message(self.index)
        return delta
    
    def __init__(self, name, factor, index):
        self.schedule = schedule(name)
        self.factor = factor
        self.index = index
        
class variable:
    
    def default_variable(self, prior):
        self.name = 'Default'
        self.value = prior
    
    def get_default(self):
        return self.value
    
    def get_key(self):
        return self.key
    
    def get_value(self):
        return self.value
    
    def keyed_variable(self, key, name, prior):
        self.name = "Variable[" + name + "]"
        self.prior = prior
        self.key = key
    
    def reset_to_prior(self, prior):
        self.value = prior
        
    def set_key(self, key):
        self.key = key
    
    def set_value(self, value):
        self.value = value
    
    def __init__(self, name, prior):
        self.name = "Variable[" + name + "]"
        self.prior = prior
        reset_to_prior()
        
class variable_factory:
    
    def create_basic_variable(self, name):
        new_var = variable(name, variable_prior_initizalizer)
        return new_var
    
    def create_keyed_variable(self, key, name):
        new_var = variable.keyed_variable(key, name, variable_prior_initizalizer)
        return new_var
    
    def __init__(self, variable_prior_initizalizer):
        self.variable_prior_initizalizer = variable_prior_initizalizer